In [4]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from tqdm import tqdm 
import os 
from glob import glob

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 로드 

In [13]:
path = '/content/drive/MyDrive/별떠라가자_공유폴더/AI파트/데이터/RAW_DATA'
csv_dir = os.listdir(path)
df_dict = {}
for dir in tqdm(csv_dir):
  df = pd.read_csv(os.path.join(path,dir))
  try:
    df.drop(columns = 'Unnamed: 0',inplace=True)
  except:
    pass
  df_dict[dir.split('.')[0]] = df 


100%|██████████| 19/19 [00:50<00:00,  2.67s/it]


In [15]:
columns = ['DATE', 'ID', 'ACCEL_X', 'ACCEL_Y', 'ACCEL_Z', 'GYRO_X', 'GYRO_Y',
       'GYRO_Z', 'PITCH', 'ROLL', 'LAT', 'LON', 'VELOCITY', 'ALT', 'EVENT0',
       'EVENT1', 'EVENT2', 'EVENT3']

## 테스트 데이터 로드 

In [16]:
test_path = '/content/drive/MyDrive/별떠라가자_공유폴더/분석데이터/블랙박스 영상 데이터'
test_dir = os.listdir(test_path)

for dir in tqdm(test_dir):
  df= pd.read_csv(os.path.join(test_path,dir),header=None)
  df.columns = columns
  df_dict[dir.split('.')[0]] = df 
id_list = list(df_dict.keys())

100%|██████████| 3/3 [00:17<00:00,  5.76s/it]


# 데이터 전처리 

### 0값 제거 

In [17]:
def remove_0(df):
  a = df.loc[(df['ACCEL_X']==0) & (df['ACCEL_Y']==0) & (df['ACCEL_Z']==0)].index
  b = df.loc[(df['GYRO_X']==0) & (df['GYRO_Y']==0) & (df['GYRO_Z']==0)].index
  c = df.loc[(df['ACCEL_X']==0) | (df['ACCEL_Y']==0) | (df['ACCEL_Z']==0)].index
  removed_index = np.unique(np.hstack([a,b,c]))
  df = df.drop(removed_index)
  return df

In [18]:
for id in id_list:
  df = df_dict[id]
  df = remove_0(df) # 0값 제거 == 결측치 제거 
  #컬럼 선별 
  df = df[['DATE','ID','ACCEL_X','ACCEL_Y','ACCEL_Z','GYRO_X','GYRO_Y','GYRO_Z']]
  df_dict[id] = df

## 타임라인 선별 
- 20개보다 작은 경우는 모두 제거 

In [19]:
def make_timestamps(temp_df):
  temp_timestamps = np.array(temp_df['DATE'].value_counts()[temp_df['DATE'].value_counts()>=19].reset_index()['index'])
  return temp_timestamps

In [20]:
def make_index(timestamp):
  global df 
  time_df = df[df['DATE']==timestamp]
  time_df_index = np.array(time_df.index)
  index_mask = np.arange(len(time_df_index))
  index_mask = np.linspace(index_mask[0],index_mask[-1],19).astype(int)
  index = time_df_index[index_mask]
  return index 

In [22]:
for id in  ['20211203','20211204','20211205']:
  
  df = df_dict[id]
  timestamps = make_timestamps(df)
  time_df_index_list = [] 

  for timestamp in tqdm(timestamps):
    index = make_index(timestamp)
    time_df_index_list.append(index)

  df= df.loc[np.unique(np.array(time_df_index_list).reshape(-1))].reset_index(drop=True)
  df_dict[id] = df 

100%|██████████| 27549/27549 [1:19:10<00:00,  5.80it/s]


In [23]:
import pickle
with open('/content/drive/MyDrive/별떠라가자_공유폴더/AI파트/데이터/학습용/df_dict.pkl','rb') as f:
  mydict = pickle.load(f)

In [24]:
for id in  ['20211203','20211204','20211205']:
  df = df_dict[id]
  mydict[id] = df
df_dict = mydict

In [25]:
import pickle 
with open('/content/drive/MyDrive/별떠라가자_공유폴더/AI파트/데이터/학습용/df_dict.pkl','wb') as f:
  pickle.dump(df_dict,f)